CÉLULA DE MARKDOWN EXEMPLO

## Projeto IIA ##
* Professor Díbio , Turma 01 , 2025.1

# Autores: #
* Marcelo Marques Rodrigues - 221018960

favor rodar{ pip install -r requirements.txt }

## Parte 1: Processar Geolocalizações e Mostrar em Mapa ##

In [30]:

import os
import pandas as pd
from geopy.distance import geodesic


Processamos o Dataset

In [31]:
caminho_arquivo_csv = os.path.join( "..", "data", "associacoes_geocoords.csv" )

try:
    df_associacoes = pd.read_csv(caminho_arquivo_csv)

except FileNotFoundError:
    print('Arquivo .csv não encontrado! Verifique o Diretório')
    df_associacoes = pd.DataFrame( columns=['Nome_Associacao', 'Localidade_Principal_Projeto', 'Latitude', 'Longitude'] )

print("Dataframe Carregado")
print(df_associacoes.head(6))
print(f'\n Número Total de Associações: {len(df_associacoes)}')

Dataframe Carregado
                                     Nome_Associacao  \
0  Associação de Agricultores Familiares da Eco C...   
1  Associação Agrícola do Distrito Federal e Ride...   
2  Associação Mista dos Agricultores Familiares, ...   
3  Associação dos Produtores da Agricultura Famil...   
4  Associação dos Produtores Rurais de Alexandre ...   
5  Associação dos Produtores de Hortigranjeiros d...   

  Localidade_Principal_Projeto  Latitude  Longitude  
0                São Sebastião -15.88007  -47.77402  
1                   Taguatinga -15.83322  -48.05677  
2                  Santa Maria -16.00157  -47.98586  
3                        Guará -15.81459  -47.98497  
4                   Brazlândia -15.67693  -48.20507  
5                         Gama -16.01609  -48.07178  

 Número Total de Associações: 17


Calculamos distância e encontramos as associações mais próximas comparadas à entrada do usuário

In [32]:
def calcular_distancia_km(coord_usuario, coord_associacao): # calculamos a distancia por coordenadas do usuario com a associação

    if None in coord_usuario or pd.isna(coord_usuario[0]) or pd.isna(coord_usuario[1]) or \
       None in coord_associacao or pd.isna(coord_associacao[0]) or pd.isna(coord_associacao[1]):
        return float('inf') # retorna infinito se a coordenada do usuária for none ou nan
    
    return geodesic(coord_usuario, coord_associacao).km

def encontrar_associacoes_proximas( df_associacoes, coord_usuario_lat, coord_usuario_lon, max_distancia_km=None, top_n=None):
    
    if df_associacoes.empty:
        print(" DataFrame vazio. Não há associação para encontrar" )
        return pd.DataFrame()

    coord_usuario = (coord_usuario_lat, coord_usuario_lon)

    df_resultado = df_associacoes.copy()

    df_resultado['Latitude'] = pd.to_numeric(df_resultado['Latitude'], errors='coerce') # tratamento
    df_resultado['Longitude'] = pd.to_numeric(df_resultado['Longitude'], errors='coerce') #tratamento
    df_resultado['Distancia_km'] = df_resultado.apply(
        lambda row: calcular_distancia_km(coord_usuario, (row['Latitude'], row['Longitude'])), # lambda permite calcular todas as dis^tanias
        axis=1
    )

    df_resultado = df_resultado.sort_values(by='Distancia_km') # sort

    if max_distancia_km is not None:
        df_resultado = df_resultado[df_resultado['Distancia_km'] <= max_distancia_km]
    if top_n is not None: # melhores N recomendações
        df_resultado = df_resultado.head(top_n)

    return df_resultado

Teste da função

In [ ]:
lat_usuario = -15.7942287 #teste meio de brasília
lon_usuario = -47.8821658


associacoes_proximas = encontrar_associacoes_proximas(
    df_associacoes,
    lat_usuario,
    lon_usuario,
    max_distancia_km= 20, # raio do filtro
)

if not associacoes_proximas.empty:
    print(f"\n Associações encontradas perto de ({lat_usuario}, {lon_usuario}):")
    colunas_para_mostrar = ['Nome_Associacao', 
                            'Localidade_Principal_Projeto',
                            'Distancia_km']
    print(associacoes_proximas[colunas_para_mostrar])
else:

    print(f"\nNenhuma associação encontrada dentro dos critérios perto do usuário.")

# Exemplo sem filtro de distância máxima, apenas as 3 mais próximas
print("\n--------------------------------------------------")
associacoes_topn = encontrar_associacoes_proximas (
    df_associacoes,
    lat_usuario,
    lon_usuario,
    top_n = 3
)

if not associacoes_topn.empty:
    print(f"\n Top 3 associações mais próximas de ({lat_usuario}, {lon_usuario}):")
    colunas_para_mostrar = ['Nome_Associacao', 'Localidade_Principal_Projeto', 'Distancia_km',]
    print(associacoes_topn[colunas_para_mostrar])
else:
    print(f"\nNenhuma associação encontrada perto do usuário.")


 Associações encontradas perto de (-15.7942287, -47.8821658):
                                      Nome_Associacao  \
11  Cooperativa dos Produtores Rurais de Planaltin...   
3   Associação dos Produtores da Agricultura Famil...   
8   Associação dos Trabalhadores Rurais da Agricul...   
10  Cooperativa Agrícola Buriti Vermelho (Cooper-H...   
0   Associação de Agricultores Familiares da Eco C...   
16  Cooperativa de Serviços Ambientais, Agricultur...   
1   Associação Agrícola do Distrito Federal e Ride...   

   Localidade_Principal_Projeto  Distancia_km  
11                 Plano Piloto      0.000040  
3                         Guará     11.242316  
8                         Guará     11.242316  
10                      Paranoá     11.268807  
0                 São Sebastião     14.981120  
16                   Sobradinho     18.374264  
1                    Taguatinga     19.197024  

--------------------------------------------------

🌟 Top 3 associações mais próximas de (-15.7